## Setup packages 

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%pip install progressbar
%pip install plotnine
%pip install torch
%pip install ipython-autotime
%load_ext autotime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for progressbar: filename=progressbar-2.5-py3-none-any.whl size=12082 sha256=f4e9bd2e6330a1b6cf2809102a404117723433564975fa9f3652b9272e500213
  Stored in directory: /root/.cache/pip/wheels/f0/fd/1f/3e35ed57e94cd8ced38dd46771f1f0f94f65fec548659ed855
Successfully built progressbar
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
time: 163 µs (started: 2022-08-02 17:01:00 +00:00)


In [4]:
from plotnine import *
from plotnine.themes import *

time: 1.77 s (started: 2022-08-02 17:01:00 +00:00)


In [5]:
import tensorflow as tf
from scipy.io import loadmat
import random
import math
import tensorflow_probability as tfp

time: 2.53 s (started: 2022-08-02 17:01:02 +00:00)


_paper_name_ establishes the reusable name of the paper, it represents the directory under data_papers on the google drive

In [6]:
paper_name = "dgm_cj_expand"

time: 785 µs (started: 2022-08-02 17:01:04 +00:00)


In [7]:
import os, sys
import errno

# make a directory if it does not exist
def make_dir_if_not_exist(used_path):
    if not os.path.isdir(used_path):
        try:
            os.mkdir(used_path)
        except OSError as exc:
            if exc.errno != errno.EEXIST:
                raise exc
            else:
                raise ValueError(f'{used_path} directoy cannot be created because its parent directory does not exist.')

# make directories if they do not exist

make_dir_if_not_exist("/content/drive/MyDrive/data_papers/")
make_dir_if_not_exist(f"/content/drive/MyDrive/data_papers/{paper_name}")
make_dir_if_not_exist(f"/content/drive/MyDrive/data_papers/{paper_name}/model_features/")
make_dir_if_not_exist(f"/content/drive/MyDrive/data_papers/{paper_name}/model_checkpoints/")
make_dir_if_not_exist(f"/content/drive/MyDrive/data_papers/{paper_name}/model_history/")
make_dir_if_not_exist(f"/content/drive/MyDrive/data_papers/{paper_name}/model_finals/")
make_dir_if_not_exist(f"/content/drive/MyDrive/data_papers/{paper_name}/model_predictions/")
make_dir_if_not_exist(f"/content/drive/MyDrive/data_papers/{paper_name}/model_ccs/")
make_dir_if_not_exist(f"/content/drive/MyDrive/data_papers/{paper_name}/summary_results/")
make_dir_if_not_exist(f"/content/drive/MyDrive/data_papers/{paper_name}/summary_results/temp/")

time: 582 ms (started: 2022-08-02 17:01:04 +00:00)


In [8]:
# Set up the imports
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np

import site
import os
import tensorflow as tf
import pandas as pd
import h5py as h5
import matplotlib.pyplot as plt
import errno
import numpy as np
import itertools
import multiprocessing
import json
import datetime
import random
from collections import defaultdict
from sklearn.model_selection import train_test_split

pd.set_option('display.width', 400)
pd.set_option('display.max_columns', 40)


time: 9.37 ms (started: 2022-08-02 17:01:05 +00:00)


In [9]:
import torch 
import matplotlib.pyplot as plt
import numpy as np
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.stats import norm
from matplotlib import cm
import pdb


time: 2.31 s (started: 2022-08-02 17:01:05 +00:00)


In [10]:
import plotly.graph_objects as go
import plotly.express as px
from pprint import pprint as pp

time: 1.84 s (started: 2022-08-02 17:01:07 +00:00)


## Shared Functions and Input Parameters across all implementations

### Shared functions across models

In [11]:
import torch

def to_cpu_detach(x):
  if isinstance(x, list):
    return [ y.detach().cpu().item() for y in x ]
  else:
    return x.detach().cpu().item()

time: 1.86 ms (started: 2022-08-02 17:01:09 +00:00)


In [12]:
def save_model_final(training, eqObject, lr, eqLossFn, sample_method, batch_size): 

  model_id_base_str =  f"{eqObject.__class__.__name__}_{datetime.datetime.now():%Y%m%d%H%M%S}_{batch_size}"
  model_id_base_str = model_id_base_str + f"_{eqLossFn}_{sample_method}_{str(lr).replace('.','p')}"  
  model_id_base_str = model_id_base_str + f"_U{eqObject.u_net.neurons}_U{eqObject.u_net.depth}_P{eqObject.pi_net.neurons}_P{eqObject.pi_net.depth}"
  
  torch.save(eqObject.u_net.state_dict(),  f"/content/drive/MyDrive/data_papers/{paper_name}/model_finals/{model_id_base_str}_U")
  torch.save(eqObject.pi_net.state_dict(), f"/content/drive/MyDrive/data_papers/{paper_name}/model_finals/{model_id_base_str}_P")

  try:
      train_losses = getattr(training,"train_losses")
      if train_losses is not None:
        train_losses.tofile(f"/content/drive/MyDrive/data_papers/{paper_name}/model_finals/trainlosses_{model_id_base_str}.csv", sep = ',')    
  except AttributeError:
      print("Class `{}` does not have `{}`".format(training.__class__.__name__, "train_losses"))
      # raise NotImplementedError("Class `{}` does not have `{}`".format(trainObj.__class__.__name__, "train_losses"))

  try:
      validation_losses = getattr(training,"validation_losses")
      if validation_losses is not None:
        validation_losses.tofile(f"/content/drive/MyDrive/data_papers/{paper_name}/model_finals/validationlosses_{model_id_base_str}.csv", sep = ',')    
  except AttributeError:
      print("Class `{}` does not have `{}`".format(training.__class__.__name__, "validation_losses"))

  # import numpy as np

  # training_dict = dict({ "BATCH_SIZE" : training.BATCH_SIZE,
  #                        "use_early_stop" : training.use_early_stop,
  #                       "early_stop_patience" : training.early_stop_patience,
  #                       "early_stop_delta" : training.early_stop_delta,
  #                       "monitored_loss_type" : training.monitored_loss_type,
  #                       "use_early_stop" : training.use_early_stop,
  #                       "stop_epoch" : training.stop_epoch            })
  # equation_dict = dict({ 
  #                       "xbreaks" : eqObject.xbreaks,
  #                       "tbreaks" : eqObject.tbreaks,
  #                       "MAX_X" : eqObject.MAX_X,
  #                       "T" : eqObject.T,
  #                       "MAX_MU" : eqObject.MAX_MU,
  #                       "MAX_SIGMA" : eqObject.MAX_SIGMA,
  #                       "pi_net_epoch" : eqObject.pi_net_epoch,
  #                       "pi_net_lr" : eqObject.pi_net_lr,
  #                       # "loss_multiply" : eqObject.loss_multiply,
  #                       "epoch_of_u" : eqObject.epoch_of_u,
  #                       "adapt_pi_epochs" : eqObject.adapt_pi_epochs,
  #                       "start_adapt_epochs" : eqObject.start_adapt_epochs,
  #                       "was_loss_beaten" : eqObject.was_loss_beaten  })

  # np.save(f"/content/drive/MyDrive/data_papers/{paper_name}/model_finals/trainingDict_{model_id_base_str}.npy", training_dict)
  # np.save(f"/content/drive/MyDrive/data_papers/{paper_name}/model_finals/equationDict_{model_id_base_str}.npy", equation_dict) 


time: 34.1 ms (started: 2022-08-02 17:01:09 +00:00)


### Shared input Parameters

In [13]:
# drift of underlying
mu = 0.0
# volatility of underlying BM
sigma = 0.01
# inventory penalty
phi = 1E-5  # function of sigma on a portfolio?!
# arrival rate/s of market orders
lambda_ask = 1.0
lambda_bid = 1.0
lambda_up = lambda_ask
lambda_down = lambda_bid
# exponential likelihood or order hit as function of depth
kappa_ask = 100.0
kappa_bid = 100.0
kappa_up = kappa_ask
kappa_down = kappa_bid
kappa = (kappa_ask+kappa_bid)/2.0
# fee for taking liquidity
alpha = 1E-4
# time-position (domain on which to solve PDE)
# Time parameters
t_low = 0 + 1e-10  # time lower bound
T = 30.0
# max postions
qbar_long = 3
qbar_short = -3
q_up = qbar_long
q_down = qbar_short
q_high = qbar_long
q_low = qbar_short

time: 11.7 ms (started: 2022-08-02 17:01:09 +00:00)


## Architectures

### Terminal Losses & Rounding Ticks

In [14]:
def terminalV(q, alpha):
    return -alpha * q ** 2

time: 657 µs (started: 2022-08-02 17:01:09 +00:00)


In [15]:
def nearest_nth(x, fraction_of_1 = 1.0/0.25):
    return round(x*fraction_of_1)/fraction_of_1

time: 923 µs (started: 2022-08-02 17:01:09 +00:00)


### NN Architectures

[implement from github for the ResNet](https://github.com/Plemeur/DGM/blob/master/first_net.py)

[Optional CrossProd layer also from github](https://johaupt.github.io/blog/cross_layer_network.html)

In [16]:
## Basic DNN structure
class BasicDNNet(nn.Module):
    def __init__(self , NL  , NN, activation = torch.tanh, init_func = torch.nn.init.xavier_uniform_):
        super(BasicDNNet, self).__init__()
        self.NL = NL
        self.NN = NN
        self.Input = 5 + 1  # wealth, time, mu, r, sigma, pi
        self.fc_input = nn.Linear(self.Input,self.NN)
        init_func(self.fc_input.weight)
        self.linears = nn.ModuleList([nn.Linear(self.NN, self.NN) for i in range(self.NL)])
        for i, l in enumerate(self.linears):    
            init_func(l.weight)
        self.fc_output = nn.Linear(self.NN,1)
        init_func(self.fc_output.weight)
        self.act = activation
        
    def forward(self, x):
        h = self.act(self.fc_input(x))
        for i, l in enumerate(self.linears):
            h = self.act( l(h) )
        out = self.fc_output(h)
        return out 

time: 11.6 ms (started: 2022-08-02 17:01:09 +00:00)


In [17]:
## Crossprod layer

class Cross(nn.Module):
    def __init__(self, input_features, 
                 batch_normalize=False,
                 kaiming_he_init = False):
        super().__init__()
        self.input_features = input_features

        self.batch_normalize = batch_normalize
        if self.batch_normalize == True:
          self.batch_norm = torch.nn.BatchNorm1d(input_features)

        self.weights = nn.Parameter(torch.Tensor(input_features))
        # Kaiming/He initialization with a=0
        if kaiming_he_init:
          nn.init.normal_(self.weights, mean=0, std=math.sqrt(2/input_features))
        else:
          nn.init.constant_(self.weights, 1.)
        
        self.bias = nn.Parameter(torch.Tensor(input_features))
        nn.init.constant_(self.bias, 0.)
        
    def forward(self, x0, x):
        x0xl = torch.bmm(x0.unsqueeze(-1), x.unsqueeze(-2))
        if self.batch_normalize == True:
          return self.batch_norm(torch.tensordot(x0xl, self.weights, [[-1],[0]]) + self.bias + x)
        return torch.tensordot(x0xl, self.weights, [[-1],[0]]) + self.bias + x
    
    # Define some output to give when layer 
    def extra_repr(self):
        return 'in_features={}, out_features={}'.format(
            self.input_features, self.input_features
        )


time: 13.1 ms (started: 2022-08-02 17:01:09 +00:00)


In [18]:
from pprint import pprint as pp
cl = Cross(10, batch_normalize=True)
input = torch.tensor([ float(x) for x in range(10)]).reshape(-1,1)
pp(type(input))
pp(input.shape)
pp(cl(input, input).shape)
pp(cl(input, input).reshape(-1,1).shape)

<class 'torch.Tensor'>
torch.Size([10, 1])
torch.Size([10, 10])
torch.Size([100, 1])
time: 127 ms (started: 2022-08-02 17:01:09 +00:00)


In [19]:
## Setting up ResNet like layers

class LinearWithXavier(nn.Module):
    """ Copy of linear module from Pytorch, modified to have a Xavier init,
        TODO : figure out what to do with the bias"""
    def __init__(self, in_features, out_features, bias=True, batch_normalize=True):
        super(LinearWithXavier, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = torch.nn.Parameter(torch.Tensor(out_features, in_features))
        self.batch_normalize = batch_normalize
        
        if self.batch_normalize == True:
          self.batch_norm = torch.nn.BatchNorm1d(out_features)
        
        if bias:
            self.bias = torch.nn.Parameter(torch.Tensor(out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()
    
    def reset_parameters(self):
        torch.nn.init.xavier_uniform_(self.weight)
        if self.bias is not None:
            torch.nn.init.uniform_(self.bias, -1, 1) #boundary matter?
    
    def forward(self, input):
        if self.batch_normalize == True:
          return self.batch_norm(torch.nn.functional.linear(input, self.weight, self.bias))
        return torch.nn.functional.linear(input, self.weight, self.bias)
    
    def extra_repr(self):
        return 'in_features={}, out_features={}, bias={}'.format(
            self.in_features, self.out_features, self.bias is not None
        )


class DGM_layer(nn.Module):
    """ See readme for paper source"""
    def __init__(self, in_features, out_feature, residual=False, batch_normalize=False):
        super(DGM_layer, self).__init__()
        self.residual = residual

        self.Z = LinearWithXavier(out_feature, out_feature, batch_normalize=batch_normalize)
        self.UZ = LinearWithXavier(in_features, out_feature, bias=False, batch_normalize=batch_normalize)
        self.G = LinearWithXavier(out_feature, out_feature, batch_normalize=batch_normalize)
        self.UG = LinearWithXavier(in_features, out_feature, bias=False, batch_normalize=batch_normalize)
        self.R = LinearWithXavier(out_feature, out_feature, batch_normalize=batch_normalize)
        self.UR = LinearWithXavier(in_features, out_feature, bias=False, batch_normalize=batch_normalize)
        self.H = LinearWithXavier(out_feature, out_feature, batch_normalize=batch_normalize)
        self.UH = LinearWithXavier(in_features, out_feature, bias=False, batch_normalize=batch_normalize)

    def forward(self, x, s):
        z = torch.tanh(self.UZ(x) + self.Z(s))
        g = torch.tanh(self.UG(x) + self.G(s))
        r = torch.tanh(self.UR(x) + self.R(s))
        h = torch.tanh(self.UH(x) + self.H(s * r))
        return (1 - g) * h + z * s


class ResNetBasedNN(nn.Module):

    def __init__(self, in_size, out_size, neurons, depth, 
                 is_softmax_out = False,
                 cross_first = False,
                 activation_func = torch.tanh):
        super(ResNetBasedNN, self).__init__()
        self.neurons=neurons
        self.depth=depth
        self.activation_func = activation_func
        self.dim = in_size

        self.cross_layer = Cross(in_size) if cross_first else None
        self.input_layer = LinearWithXavier(in_size*in_size, neurons) if cross_first else LinearWithXavier(in_size, neurons)
        self.middle_layer = nn.ModuleList([DGM_layer(in_size, neurons) for i in range(depth)])
        # self.middle_layer_2 = nn.ModuleList([DGM_layer(in_size, neurons, batch_normalize=False) for i in range(2)])
        self.final_layer = LinearWithXavier(neurons, out_size, batch_normalize=False) if not is_softmax_out else torch.nn.Softmax(neurons, out_size)

    def forward(self, X):
        s = None
        
        if self.cross_layer:
          s = self.activation_func(self.input_layer(self.cross_layer(X,X)))
        else:
          s = self.activation_func(self.input_layer(X))   

        for i, layer in enumerate(self.middle_layer):
            s = self.activation_func(layer(X, s))
        
        # s = torch.nn.functional.gelu(self.input_layer(X))
        # s = torch.nn.functional.elu(layer(X, s))

        return self.final_layer(s)
        # return torch.pow(self.final_layer(s), 1)


time: 97.9 ms (started: 2022-08-02 17:01:09 +00:00)


### Equation Classes

In [20]:
pp(torch.round(torch.tensor([-1.4, -1.45, -1.95, 3.32, 3.54, 3.69, 3.78])).int())
pp(torch.tensor([-1.4, -1.45, -1.95, 3.32, 3.54, 3.69, 3.78]).int())

tensor([-1, -1, -2,  3,  4,  4,  4], dtype=torch.int32)
tensor([-1, -1, -1,  3,  3,  3,  3], dtype=torch.int32)
time: 21.4 ms (started: 2022-08-02 17:01:09 +00:00)


In [21]:
# https://www.abebooks.co.uk/products/isbn/9781107091146

import math
from functools import partial

class CJEquation10p4():
    
    def __init__(self , 
                 u_net, 
                 stock_start = 1.0, 
                 tick_size = 0.0025, 
                 max_q = 10.0, 
                 num_of_deltas = 3,
                 term_utility_function = terminalV):

        self.u_net = u_net
        
        self.pi_net_epoch = 1
        self.epoch_of_u = None
        self.adapt_pi_epochs = False
        # self.start_adapt_epochs = [ [500, 1000, 2000, 5000, 10000, 15000 ], [4, 8, 10 ,20, 40, 60] ]
        self.start_adapt_epochs = [ [15000], [60] ]
        self.was_loss_beaten = False

        # self.wgamma = 0.0001
        # self.term_utility_func = term_utility_function
        
        self.loss_multiply = 1.0

        self.S_MULT = 1.6  # worst case is 60% loss, best case is 60% profit
        self.PHI_MIN = 1e-6    # sample this on a reverse log scale perhaps...
        self.PHI_MAX = 0.1
        self.LAMBDA_MIN = 0.01 # sample this on a reverse log scale perhaps...
        self.LAMBDA_MAX = 10.0
        self.KAPPA_MIN = 1.0  # sample this on a reverse log scale perhaps...
        self.KAPPA_MAX = 1000.0
        self.ALPHA_MIN = 1e-6  # sample this on a reverse log scale perhaps...
        self.ALPHA_MAX = 1.0
        self.T_MIN = 0 + 1e-10
        self.T_MAX = 1.0
        self.QBAR_ULT = max_q
        self.NUM_DELTAS = num_of_deltas
        self.S_START = stock_start
        self.S_TICK = tick_size  # 40 ticks is 10% of the stock...
        self.X_MIN_MULT = -max_q # assuming your min cash position is your worst position x the stock's value...
        self.X_MAX_MULT = max_q # assuming your max cash position is your worst position x the stock's value...

        self.SIGMA_FIXED = 0.2
        self.stock_round = partial(nearest_nth, fraction_of_1 = 1.0/self.S_TICK)


    def g(self,x):
        # Time, Wealth, Mu, R, Sigma
        return self.term_utility_func(x[:,1].reshape(-1,1),x[:,7].reshape(-1,1))

    @staticmethod
    def to_device(x, to_cpu):
      if to_cpu:
        return x.cpu()
      else:
        return x.cuda()

    def sample_parameter(self, size, min_value, max_value, apply_log = False):
        if not apply_log:
          return min_value + (max_value-min_value)*torch.rand([size, 1])
        else:
          return torch.exp(np.log(min_value)  + (np.log(max_value)-np.log(min_value))*torch.rand([size, 1]))

    def sample(self , size = 2**8, to_cpu = False, 
               sample_log_scale = [False, True, True, True, True, True, True, False, False, False, False] ):
        '''
        Sampling function

        ## sampling x dimesions : nsamples x nofparameters
        # 0 = stock
        # 1 = carry inventory penalty (phi)
        # 2 = arrival rate/s of market orders on the ask (lambda_ask)
        # 3 = arrival rate/s of market orders on the bid (lambda_bid)
        # 4 = exponential likelihood or order hit as function of depth on the ask (kappa_ask)
        # 5 = exponential likelihood or order hit as function of depth on the bid (kappa_bid)
        # 6 = fee for taking liquidity at the final stage... (alpha)
        # 7 = time (domain on which to solve PDE)
        # 8 = max long position (qbar_long) 
        # 9 = max short position (qbar_short) as a negative number
        # 10 = x past cash position (could be positive or negative...)
        # 11 = q current inventory
        '''
        qbar_long_sample = torch.round(self.sample_parameter(size, 1.0, self.QBAR_ULT, sample_log_scale[8]), decimals=0).int()
        # sample_log_scale[10] should always be False 
        q_sample_internal = torch.round(self.sample_parameter(size, -qbar_long_sample, qbar_long_sample, False)).int()

        stock_sample_internal = None
        if sample_log_scale[0]:
          # we will sample as many up stocks as down stocks... (cluster around the start)
          stock_sample_internal = torch.cat((self.S_START * self.sample_parameter(size/2, 1.0, self.S_MULT, True),
                                             self.S_START * (2.0 - (self.sample_parameter(size/2, 1.0, self.S_MULT, True))) ), 
                                            dim = 0 )
        else:
          stock_sample_internal = self.S_START * self.sample_parameter(size, 2.0-self.S_MULT, self.S_MULT, False)

        x_internal = self.to_device(torch.cat(( stock_sample_internal,
                                                self.sample_parameter(size, self.PHI_MIN, self.PHI_MAX, sample_log_scale[1]),                 
                                                self.sample_parameter(size, self.LAMBDA_MIN, self.LAMBDA_MAX, sample_log_scale[2]),                 
                                                self.sample_parameter(size, self.LAMBDA_MIN, self.LAMBDA_MAX, sample_log_scale[3]),  
                                                self.sample_parameter(size, self.KAPPA_MIN, self.KAPPA_MAX, sample_log_scale[4]),   
                                                self.sample_parameter(size, self.KAPPA_MIN, self.KAPPA_MAX, sample_log_scale[5]),   
                                                self.sample_parameter(size, self.ALPHA_MIN, self.ALPHA_MAX, sample_log_scale[6]),   
                                                self.sample_parameter(size, self.T_MIN, self.T_MAX, sample_log_scale[7]),   
                                                qbar_long_sample,
                                                -qbar_long_sample,
                                                self.S_START-self.sample_parameter(size, 
                                                                                   self.X_MIN_MULT*self.S_START, 
                                                                                   self.X_MAX_MULT*self.S_START, 
                                                                                   sample_log_scale[9]),
                                               q_sample_internal                                               
                                        ) , dim = 1 ), to_cpu)
        qbar_long_sample_terminal = torch.round(self.sample_parameter(size, 1.0, self.QBAR_ULT, sample_log_scale[8]),decimals=0).int()
        q_sample_terminal = torch.round(self.sample_parameter(size, -qbar_long_sample_terminal, qbar_long_sample_terminal, False)).int()

        stock_sample_terminal = None
        if sample_log_scale[0]:
          # we will sample as many up stocks as down stocks... (cluster around the start)
          stock_sample_terminal = torch.cat((self.S_START * self.sample_parameter(size/2, 1.0, self.S_MULT, True),
                                             self.S_START * (2.0 - (self.sample_parameter(size/2, 1.0, self.S_MULT, True))) ), 
                                            dim = 0 )
        else:
          stock_sample_terminal = self.S_START * self.sample_parameter(size, 2.0-self.S_MULT, self.S_MULT, False)

        x_terminal = self.to_device(torch.cat(( stock_sample_terminal,
                                                self.sample_parameter(size, self.PHI_MIN, self.PHI_MAX, sample_log_scale[1]),                 
                                                self.sample_parameter(size, self.LAMBDA_MIN, self.LAMBDA_MAX, sample_log_scale[2]),                 
                                                self.sample_parameter(size, self.LAMBDA_MIN, self.LAMBDA_MAX, sample_log_scale[3]),  
                                                self.sample_parameter(size, self.KAPPA_MIN, self.KAPPA_MAX, sample_log_scale[4]),   
                                                self.sample_parameter(size, self.KAPPA_MIN, self.KAPPA_MAX, sample_log_scale[5]),   
                                                self.sample_parameter(size, self.ALPHA_MIN, self.ALPHA_MAX, sample_log_scale[6]),  
                                                self.T_MAX*torch.ones([size, 1]),                                                 
                                                qbar_long_sample_terminal,
                                                -qbar_long_sample_terminal,
                                                self.S_START-self.sample_parameter(size, 
                                                                                   self.X_MIN_MULT*self.S_START, 
                                                                                   self.X_MAX_MULT*self.S_START, 
                                                                                   sample_log_scale[9]),
                                               q_sample_terminal
                                        ) , dim = 1 ),to_cpu)
        
        # x_initial = torch.cat( ( torch.zeros(size, 1), -self.MAX_X*range_multiplier*torch.rand([size, 1])+self.MAX_X*range_multiplier) , dim = 1 ).cuda()
        return x_internal , x_terminal


    def get_num_pi_epochs(self):
      if self.adapt_pi_epochs:
        if any([ i < self.epoch_of_u for i in self.start_adapt_epochs[0] ]):
          return self.pi_net_epoch * self.start_adapt_epochs[1][[i for i in range(len(self.start_adapt_epochs[0])) if self.start_adapt_epochs[0][i] < self.epoch_of_u][-1]] 
      return self.pi_net_epoch

    
    def get_utility_function_derivatives(self, u_net_val, x_internal, normalize=False):
        
        du = torch.autograd.grad( u_net_val, 
                                  x_internal, 
                                  grad_outputs=torch.ones_like(u_net_val),
                                  create_graph=True,
                                  retain_graph=True)
        
        du_dt = du[0][:,7].reshape(-1,1)
        du_dx = du[0][:,0].reshape(-1,1)     

        d2u_dx2 = torch.autograd.grad(du_dx, 
                                      x_internal , 
                                      grad_outputs=torch.ones_like(du_dx),
                                      retain_graph=True
                                      )[0][:,0].reshape(-1,1)
        
        # d2u_dx2 = torch.sign(d2u_dx2) * torch.maximum(torch.abs(d2u_dx2), 1e-8*torch.ones_like(d2u_dx2))                                      

        return du_dt, du_dx, d2u_dx2 
   
    def criterion(self, x_internal, x_terminal, loss_transform = torch.abs, util_network=None):
        '''
        Loss function that helps network find solution to equation
        '''   
        # mu / sigma / phi / lambda_bid / lambda_ask / kappa_bid / kappa_ask / alpha / time of day / qmax long / qmin short (sample data order)
        x_internal_before = x_internal.clone()
        x_terminal_before = x_terminal.clone()

        x_internal =  Variable(x_internal, requires_grad=True)
        x_terminal =  Variable(x_terminal, requires_grad=True)
        
        if util_network is None:
          util_network = self.u_net

        u_net_val = util_network(x_internal)
        u_net_val_terminal = util_network(x_terminal)
       
        du_dt, du_dx, d2u_dx2 = self.get_utility_function_derivatives(u_net_val, x_internal)
        # du_dt_terminal, du_dx_terminal, d2u_dx2_terminal = self.get_utility_function_derivatives(u_net_val_terminal, x_terminal)
        # du_dt = (du_dt + du_dt_terminal)/2.0
        # du_dx = (du_dx + du_dx_terminal)/2.0
        # d2u_dx2 = (d2u_dx2 + d2u_dx2_terminal)/2.0

        val_func_non_opt = du_dt + \
                            0.5*self.SIGMA_FIXED*self.SIGMA_FIXED*d2u_dx2 - \
                            x_internal[:,1].reshape(-1,1)*torch.square(x_internal[:,10].reshape(-1,1)) 

        # here we train control only once per self.num_of_value_iters training epochs of value net
        control_model_both = ControlEquationBothSidesDeltas(self.control_net, 
                                                      self.util_network, 
                                                      self.q_low, 
                                                      self.q_high, 
                                                      grads)

        control_model_both.curr_epoch = self.epoch_of_u

        #control_model_minus.curr_epoch = self.epoch_of_u
        
        control_trainer = TrainInternalControl(self, 
                                               control_model_both, 
                                                x_internal.shape[0], 
                                                self.get_num_control_epochs(), 
                                                self.control_net_lr,
                                                hook_interval=self.hook_interval, sync_flag=self.sync_flag)           
        control_trainer.train(sample_batch=(x_internal, x_terminal))

        delta_plus_used, delta_minus_used = self.control_net(x_internal)

        ## sampling x dimesions : nsamples x nofparameters
        # 0 = stock
        # 1 = carry inventory penalty (phi)
        # 2 = arrival rate/s of market orders on the ask (lambda_ask)
        # 3 = arrival rate/s of market orders on the bid (lambda_bid)
        # 4 = exponential likelihood or order hit as function of depth on the ask (kappa_ask)
        # 5 = exponential likelihood or order hit as function of depth on the bid (kappa_bid)
        # 6 = fee for taking liquidity at the final stage... (alpha)
        # 7 = time (domain on which to solve PDE)
        # 8 = max long position (qbar_long) 
        # 9 = max short position (qbar_short) as a negative number
        # 10 = x past cash position (could be positive or negative...)
        # 11 = q current inventory

        q = x_internal[:,3].reshape(-1, 1)
        sigma = x_internal[:,4].reshape(-1, 1)
        phi = x_internal[:,5].reshape(-1, 1)
        kappa_plus = x_internal[:,7].reshape(-1, 1)
        kappa_minus = x_internal[:,8].reshape(-1, 1)
        lamda_plus = x_internal[:,9].reshape(-1, 1)
        lamda_minus = x_internal[:,10].reshape(-1, 1)

        batch_size = x_internal.shape[0]
          
        if loss_transforms is None:
          loss_transforms = [torch.square]

        intC = None
        terC = None

        if len(x_internal) == 0:
          intC = [ torch.tensor(0).cuda().float() for loss_transform in loss_transforms ] 
        else:
          

          control_preds = self.control_net_plus(x_internal)



        delta_plus_options =  [ -(i+1)*self.S_TICK for i in range(self.NUM_DELTAS)].append(0.0)
        
        
                              [ (i+1)*self.S_TICK for i in range(self.NUM_DELTAS) ]

        delta_plus = x_internal[:,2].reshape(-1,1)





        if loss_transform is None:
          loss_transform = torch.abs

        intC = None
        terC = None

        if len(x_internal) == 0:
          intC = torch.tensor(0).cuda().float() 
        else:
          # # pdb.set_trace()
          # pi_net_preds = self.pi_net_target(x_internal_before)
          # pi_net_preds = pi_net_preds.detach().reshape(-1,1)
          
          # x_internal =  Variable(x_internal_before.clone(), requires_grad=True)
          
          # u_net_val = util_network(x_internal)
          # du_dt, du_dx, d2u_dx2 = self.get_utility_function_derivatives(u_net_val, x_internal)

          x_internal_const = x_internal.detach().clone()
          # x_internal_const = x_internal.clone()
          
          # pi*(mu-r)
          intC_loss_1 = pi_net_preds*(x_internal_const[:,2].reshape(-1,1)-x_internal_const[:,3].reshape(-1,1)) 
          
          # r * wealth
          intC_loss_2 =  x_internal_const[:,3].reshape(-1,1)*x_internal_const[:,1].reshape(-1,1)  
          
          # sigma^2 * pi^2
          intC_loss_3 = (x_internal_const[:,4].reshape(-1,1)**2)*(pi_net_preds**2)

          intC_loss = du_dt + (intC_loss_1 + intC_loss_2)*du_dx + 0.5*intC_loss_3*d2u_dx2

          # eps = 1e-3
          # intC_loss = torch.abs((intC_loss-eps)/eps)
          intC = [ loss_transform(intC_loss) for loss_transform in loss_transforms ] 
        
        
        # print('mu:', x_internal[:,2].reshape(-1,1))
        # print('r:', x_internal[:,3].reshape(-1,1))
        # print('wealth:', x_internal[:,1].reshape(-1,1))
        # print('sigma^2', x_internal[:,4].reshape(-1,1)**2)
        # print('pi:', pi_net_preds)

        # Terminal Condition - should be equal (both in- and out of the money)
        x_terminal_before = x_terminal.clone()

        x_terminal =  Variable(x_terminal_before, requires_grad=True)
        # print('non terminal pred', u_net_val)
        # print('terminal utility pred', util_network(x_terminal))
        # print('utility target', self.g(x_terminal))
        # print('----------------------')
        # print('g terminal:', self.g(x_terminal))
        # print('unet:', util_network(x_terminal))
        # print('diff:', (self.g(x_terminal)-util_network(x_terminal)) )
        # print('perecentage error:', (self.g(x_terminal)-util_network(x_terminal))/(self.g(x_terminal) + 1e-7))
        # print('------------------------------')
        
        terminal_target = self.g(x_terminal)
        util_net_pred = util_network(x_terminal)

        err1 = torch.abs((util_net_pred-terminal_target)/(util_net_pred+1e-7) )
        err2 = torch.abs((util_net_pred-terminal_target)/(terminal_target+1e-7) )
        terC = torch.where(torch.abs(terminal_target) < torch.abs(util_net_pred), err2, err1)

        terC = terminal_target - util_net_pred

        terC = [ loss_transform(terC) for loss_transform in loss_transforms ]
        
        # print('mean d2u_dx2:', d2u_dx2.mean(),'max d2u_dx2:', d2u_dx2.max())
        # print('mean du_dx:', du_dx.mean(),'max du_dx:', d2u_dx2.max())
        # print('mean du_dt:', du_dt.mean(),'max d2u_dx2:', d2u_dx2.max())
        
        return  intC , terC

    def calculateLoss(self, batch_x, train = True, loss_transform = torch.abs, keep_batch = False, util_network=None):
        '''
        Helper function that Sample and Calculate loss,
        '''        
        # pdb.set_trace()
        x_internal , x_terminal = batch_x
        x_internal = Variable( x_internal , requires_grad=True)
        x_terminal = Variable( x_terminal , requires_grad=True)
        # print('MertonEquation calling self.criterion')

        if util_network is None:
          util_network = self.u_net

        Ls = self.criterion(x_internal , x_terminal, loss_transform = loss_transform, util_network=util_network)
        intC , terC  = Ls

        return_losses = []
        # print('internal Loss', torch.mean(intC[0]))
        # print('external Loss', torch.mean(terC[0]))

        for lc in range(len(loss_transforms)):
          if not keep_batch:
            loss_equalWeightedByType = torch.mean(intC[lc]) + torch.mean(terC[lc])
            return_losses.append( [ loss_equalWeightedByType , 
                                    torch.mean(intC[lc]) , torch.mean(terC[lc]), 
                                    loss_equalWeightedByType ] )            
          else:
            return_losses.append( [intC.numpy(), terC.numpy()] )
        return return_losses

    # def calculateLossKLMinMaxGamma(self , batch_x , train = True, loss_transforms = [ torch.square ], keep_batch = False):
    #     '''
    #     Helper function that Samples and Calculate loss,
    #     This is adapted in that it changes the weights on the losses
    #     and the distribution of sampling to maximize the loss provided 
    #     the KL distance of the loss is within positive constraints
    #     beta represents the constraints on the weights
    #     gamma represents the constraints on the sampling distribution
    #     (each representing an upper bound the KL distribution)
    #     '''        
    #     # x , x_terminal , x_initial = self.sample(sample_method_X, size)
    #     x , x_terminal  = batch_x
    #     x = Variable( x, requires_grad=True)
    #     Ls = self.criterion( x , x_terminal, loss_transforms = loss_transforms)
    #     intC , terC  = Ls

    #     if self.weights is None:
    #       self.weights = torch.ones(1,len(Ls)).to(intC[0].device)/len(Ls)
        
    #     return_losses = []
    #     for lc in range(len(loss_transforms)):
    #       if not keep_batch:
    #         intCt = self.weights[0,0] * (1.0/intC[lc].numel() if intC[lc].numel() > 0 else 0.0) * torch.sum( torch.exp(self.gamma * intC[lc])) 
    #         terCt = self.weights[0,1] * (1.0/terC[lc].numel() if terC[lc].numel() > 0 else 0.0) * torch.sum( torch.exp(self.gamma * terC[lc])) 
    #         loss_equalWeightedByType = 0.5*torch.mean(intC[lc]) + 0.5*torch.mean(terC[lc]) 
    #         transformed_loss = 1.0/self.gamma * torch.log(intCt + terCt )
    #         return_losses.append( [ transformed_loss , 
    #                                 0.5*torch.mean(intC[lc]) , 0.5*torch.mean(terC[lc]),
    #                                 loss_equalWeightedByType ] )            
    #       else:
    #         return_losses.append( [intC[lc].numpy(), terC[lc].numpy()] )
    #     return return_losses
    

IndentationError: ignored

In [23]:
class ControlEquationBothSidesDeltas:

    def __init__(self,  control_net, 
                        value_net, 
                        delta_minus_max, 
                        delta_plus_max, 
                        max_position, 
                        value_func_no_control_part = None,
                        curr_epoch=0): 
                        # grads, 
        
        using_fixed_deltas = False
        if isinstance(delta_minus_max, list) &  isinstance(delta_plus_max, list):
          using_fixed_deltas = True
        elif not isinstance(delta_minus_max, list) and not isinstance(delta_plus_max, list):
          using_fixed_deltas = False
        else:
          raise ValueError("deltas must be either both numeric or both numeric lists")

        # infinity_delta = 
        self.control_net = control_net
        self.value_net = value_net
        self.curr_epoch = curr_epoch
        self.q_bar_max_long = max_position
        self.q_bar_min_short = -max_position
        self.value_func_no_control_part = value_func_no_control_part
        # self.grads = grads

        self.delta_minus_max = delta_minus_max
        self.delta_plus_max = delta_plus_max
        self.delta_combinations = torch.tensor([(x,y) for x in self.delta_minus_max for y in self.delta_plus_max ]) if using_fixed_deltas else None
        self.using_fixed_deltas = using_fixed_deltas


    def criterion(self, x_internal, fixed_value_func = None, loss_transform = torch.abs):
      
        intC_loss = None
        if len(x_internal) == 0:
          intC_loss = torch.tensor(0).cuda().float()  
        else:
          # x_internal
          ## sampling x dimesions : nsamples x nofparameters
          # 0 = stock
          # 1 = carry inventory penalty (phi)
          # 2 = arrival rate/s of market orders on the ask (lambda_ask)
          # 3 = arrival rate/s of market orders on the bid (lambda_bid)
          # 4 = exponential likelihood or order hit as function of depth on the ask (kappa_ask)
          # 5 = exponential likelihood or order hit as function of depth on the bid (kappa_bid)
          # 6 = fee for taking liquidity at the final stage... (alpha)
          # 7 = time (domain on which to solve PDE)
          # 8 = max long position (qbar_long) 
          # 9 = max short position (qbar_short) as a negative number        
          # 10 = x past cash position (could be positive or negative...)
          # 11 = q current inventory
          if fixed_value_func is None:
            fixed_value_func = self.value_func_no_control_part

          control_preds = self.control_net(x_internal)  

          delta_plus_used = None
          delta_minus_used = None
          if self.using_fixed_deltas:
            control_pred_idx = torch.argmax(control_preds, dim=0) # picks the sup delta plus and sup delta minus
            delta_plus_used, delta_minus_used  = [ torch.tensor(x) for x in self.delta_combinations[control_pred_idx,:].reshape(-1,1).tolist() ] 
          else:
            delta_plus_used, delta_minus_used = control_preds  

          q_starting_inventory = x_internal[:,11].reshape(-1, 1)

          x_internal_for_both_deltas = x_internal.detach().clone() 
          x_internal_for_both_deltas.requires_grad = False # should already be the case..
          baseH_for_both = self.value_net(x_internal_for_both_deltas)

          x_internal_for_delta_plus = x_internal.detach().clone()
          x_internal_for_delta_plus.requires_grad = False # should already be the case..
          x_internal_for_delta_plus[:,10] = x_internal_for_delta_plus[:,10] + x_internal_for_delta_plus[:,0] + delta_plus_used
          x_internal_for_delta_plus[:,11] = x_internal_for_delta_plus[:,11] - 1
          hValue_for_delta_plus = self.value_net(x_internal_for_delta_plus)

          x_internal_for_delta_minus = x_internal.detach().clone()
          x_internal_for_delta_minus.requires_grad = False # should already be the case..
          x_internal_for_delta_minus[:,10] = x_internal_for_delta_minus[:,10] - x_internal_for_delta_plus[:,0] - delta_minus_used
          x_internal_for_delta_minus[:,11] = x_internal_for_delta_minus[:,11] + 1
          hValue_for_delta_minus = self.value_net(x_internal_for_delta_minus)

          lambda_plus = x_internal[:,2].reshape(-1, 1)
          lambda_minus = x_internal[:,3].reshape(-1, 1)

          kappa_plus = x_internal[:,4].reshape(-1, 1)
          kappa_minus = x_internal[:,5].reshape(-1, 1)

          delta_plus_control_contrib =  lambda_plus * (q_starting_inventory > self.q_bar_min_short) * torch.ones_like(q_starting_inventory) * \
                                        torch.exp(-kappa_plus * delta_plus_used) * (hValue_for_delta_plus - baseH_for_both)

          delta_minus_control_contrib =  lambda_minus * (q_starting_inventory < self.q_bar_max_long) * torch.ones_like(q_starting_inventory) * \
                                         torch.exp(-kappa_minus * delta_minus_used) * (hValue_for_delta_minus - baseH_for_both)
          
          intC_loss = fixed_value_func + delta_plus_control_contrib + delta_minus_control_contrib
          intC_loss = loss_transform(intC_loss)

        return intC_loss

    def calculateControlLoss(self, x_internal, keep_batch = False, fixed_value_func = None, loss_transform = torch.abs):
        '''
        Helper function that Sample and Calculate loss,
        '''        
        Ls = self.criterion(x_internal, fixed_value_func = fixed_value_func, loss_transform = loss_transform)
        
        if not keep_batch:          
          return torch.mean(Ls)           
        else:
          return Ls


time: 86.7 ms (started: 2022-08-02 17:03:43 +00:00)


In [ ]:
# a = [1.2, 4.3, 5.3, 3.2]
# b = [3.4, 1.2, 32.2, 0.5]
# # torch.ones_like(torch.tensor(a)) * (torch.tensor(a) > torch.tensor(b))
# (torch.tensor(a) > 2.0)

tensor([False,  True,  True,  True])

time: 5.74 ms (started: 2022-07-31 21:47:11 +00:00)


### Training Architectures

In [ ]:
class TrainBothInternalControls():
    
    def __init__(self , 
                 u_equation, 
                 pi_equation,
                 BATCH_SIZE, 
                 epoch, 
                 lr, 
                 debug = False, 
                 loss_multiply = 1.0):
        
        self.BATCH_SIZE = BATCH_SIZE
        self.u_model = u_equation        
        self.pi_model = pi_equation
        
        self.debug = debug  
        self.hook_interval = 20      

        self.optimizer_used = optim.Adam

        self.use_early_stop = False
        self.early_stop_patience = 10
        self.early_stop_delta = 0.0        
        self.best_loss = np.Inf
        self.early_stop_counter = 0

        self.stop_epoch = 0

        self.validation_sample = None
        self.validation_losses = None
        self.train_losses = None        

        self.epoch = epoch
        self.lr = lr

        self.loss_multiply = loss_multiply

    def train(self, eqLossFn = 'calculateControlLoss', passed_internal_sample = None, loss_transform = torch.abs):

        if self.validation_sample is not None:
          self.validation_losses = np.ones((self.epoch, 1 ), dtype='float32') * np.nan
        
        self.train_losses = np.ones((self.epoch, 2 ), dtype='float32') * np.nan

        optimizer = self.optimizer_used(self.pi_model.pi_net.parameters(), self.lr)
        # optimizer.zero_grad()
        loss_calc_method = None
        try:
            loss_calc_method = getattr(self.pi_model, eqLossFn)
        except AttributeError:
            raise NotImplementedError("Class `{}` does not implement `{}`".format(self.pi_model.__class__.__name__, eqLossFn))
        
        for e in range(self.epoch):
          sample_batch = self.u_model.sample(sample_method_X = sample_method_X, size=self.BATCH_SIZE) if passed_internal_sample is None else passed_internal_sample
          # pdb.set_trace()
          x_internal = Variable(sample_batch[0], requires_grad=True)
          x_terminal =  Variable(sample_batch[1], requires_grad=True)

          loss_internal  = loss_calc_method(x_internal, 
                                            keep_batch = False, 
                                            fixed_value_func = self.pi_equation.value_func_no_control_part,
                                            loss_transform = loss_transform) 
          #  fixed_value_func will get defaulted to this anyway, this is just to make it explicit
          loss_terminal  = loss_calc_method(x_terminal, 
                                            keep_batch = False, 
                                            fixed_value_func = self.pi_equation.value_func_no_control_part,
                                            loss_transform = loss_transform) 
          # this is where we could consider the terminal loss as well in adjusting the gradients...

          self.train_losses[e,:] = [ to_cpu_detach(loss_internal), to_cpu_detach(loss_terminal) ]

          if self.debug == True and (self.validation_sample is not None):
            loss_validation = loss_calc_method( self.validation_sample, 
                                                  keep_batch = False,
                                                  loss_transform = loss_transform )
            self.validation_losses[e,:] = [ to_cpu_detach(loss_validation) ]
          
          if self.use_early_stop:
            loss_to_check = loss
            if loss_to_check < (self.best_loss-self.early_stop_delta):
              self.best_loss = loss_to_check
              self.early_stop_counter = 0
            else:
              self.early_stop_counter += 1
            if self.early_stop_counter>=self.early_stop_patience:
              # print(f"Pi Early Stop at epoch {e}: {loss_to_check} with patience {self.early_stop_patience}")
              break

          optimizer.zero_grad()
          # print('self.epoch:', e)
          loss.backward()
          optimizer.step()
          
          if (e % self.hook_interval == (self.hook_interval-1)) or e == 0:
              print("Pi Epoch {} - lr {} -  key loss: {}".format(e , self.lr , float(loss.item())))

          self.stop_epoch = e
          # self.pi_model_traget.pi_net.load_state_dict(self.pi_model.pi_net.state_dict())

    def hook_fn(self, m, i, o):
              self.hooks[m] = o.detach()
            
    def get_all_layers(self, net):
      for name, layer in net._modules.items():
          if isinstance(layer, nn.ModuleList):
              for n , l in layer.named_children():
                l.register_forward_hook(self.hook_fn)
          else:
              # it's a non sequential. Register a hook
              layer.register_forward_hook(self.hook_fn)


In [22]:
class TrainCJWithDGM():
    
    def __init__(self , net , equation , BATCH_SIZE , debug = False):
        
        self.history_mean_hooks = [] 
        self.history_surfaces_hooks = None       
        self.history_tl = []
        self.history_internal = []
        self.history_terminal = []
        self.history_initial = []              
        self.history_nonzero = []
        self.BATCH_SIZE = BATCH_SIZE
        self.net = net
        self.model = equation        
        self.debug = debug  
        self.hook_interval = 20      
        if self.debug == True:
            self.hooks = {}            
            self.get_all_layers(self.net)

        self.optimizer_used = optim.Adam

        self.use_early_stop = False
        self.early_stop_patience = 10
        self.early_stop_delta = 0.0        
        self.best_loss = np.Inf
        self.monitored_loss_type = "Train_L2"
        self.early_stop_counter = 0
        self.stop_epoch = 0
        self.validation_sample = None
        self.validation_losses = None
        self.train_losses = None        
        
        self.net.train()

    def train(self, eqLossFn = 'calculateLoss', passed_internal_sample = None, loss_transform = torch.abs):

        if self.validation_sample is not None:
          self.validation_losses = np.ones((self.epoch, 1 ), dtype='float32') * np.nan
        
        self.train_losses = np.ones((self.epoch, 2 ), dtype='float32') * np.nan

        optimizer = self.optimizer_used(self.pi_model.pi_net.parameters(), self.lr)
        # optimizer.zero_grad()
        loss_calc_method = None
        try:
            loss_calc_method = getattr(self.pi_model, eqLossFn)
        except AttributeError:
            raise NotImplementedError("Class `{}` does not implement `{}`".format(self.pi_model.__class__.__name__, eqLossFn))
        
        for e in range(self.epoch):
          sample_batch = self.u_model.sample(sample_method_X = sample_method_X, size=self.BATCH_SIZE) if passed_internal_sample is None else passed_internal_sample
          # pdb.set_trace()
          x_internal = Variable(sample_batch[0], requires_grad=True)
          x_terminal =  Variable(sample_batch[1], requires_grad=True)

          loss_internal  = loss_calc_method(x_internal, 
                                            keep_batch = False, 
                                            fixed_value_func = self.pi_equation.value_func_no_control_part,
                                            loss_transform = loss_transform) 
          #  fixed_value_func will get defaulted to this anyway, this is just to make it explicit
          loss_terminal  = loss_calc_method(x_terminal, 
                                            keep_batch = False, 
                                            fixed_value_func = self.pi_equation.value_func_no_control_part,
                                            loss_transform = loss_transform) 
          # this is where we could consider the terminal loss as well in adjusting the gradients...

          self.train_losses[e,:] = [ to_cpu_detach(loss_internal), to_cpu_detach(loss_terminal) ]

          if self.debug == True and (self.validation_sample is not None):
            loss_validation = loss_calc_method( self.validation_sample, 
                                                  keep_batch = False,
                                                  loss_transform = loss_transform )
            self.validation_losses[e,:] = [ to_cpu_detach(loss_validation) ]
          
          if self.use_early_stop:
            loss_to_check = loss
            if loss_to_check < (self.best_loss-self.early_stop_delta):
              self.best_loss = loss_to_check
              self.early_stop_counter = 0
            else:
              self.early_stop_counter += 1
            if self.early_stop_counter>=self.early_stop_patience:
              # print(f"Pi Early Stop at epoch {e}: {loss_to_check} with patience {self.early_stop_patience}")
              break

          optimizer.zero_grad()
          # print('self.epoch:', e)
          loss.backward()
          optimizer.step()
          
          if (e % self.hook_interval == (self.hook_interval-1)) or e == 0:
              print("Pi Epoch {} - lr {} -  key loss: {}".format(e , self.lr , float(loss.item())))

          self.stop_epoch = e
          # self.pi_model_traget.pi_net.load_state_dict(self.pi_model.pi_net.state_dict())

    def hook_fn(self, m, i, o):
              self.hooks[m] = o.detach()
            
    def get_all_layers(self, net):
      for name, layer in net._modules.items():
          if isinstance(layer, nn.ModuleList):
              for n , l in layer.named_children():
                l.register_forward_hook(self.hook_fn)
          else:
              # it's a non sequential. Register a hook
              layer.register_forward_hook(self.hook_fn)
    
    def create_result_df(self, e, xsample, xsample_res, sample_type):
      df_xsample = pd.DataFrame(xsample.cpu().numpy(), columns = ["Time", "S1", "Mu", "R", "Sigma"])
      df_xsample["Epoch"] = e
      df_xsample["Sample"] = sample_type
      df_xsample["Result"] = xsample_res.cpu().numpy()
      return df_xsample



time: 104 ms (started: 2022-08-02 17:02:38 +00:00)


In [ ]:
# a = list(range(4))
# b = np.linspace(10.0, 14.0,5)
# pp((a,b))
# torch.tensor([ (x,y) for x in a for y in b]).shape

([0, 1, 2, 3], array([10., 11., 12., 13., 14.]))


torch.Size([20, 2])

time: 6.73 ms (started: 2022-07-30 13:28:59 +00:00)
